In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
player_path = 'gdrive/My Drive/nba_summary_17_19.csv'
team_path = 'gdrive/My Drive/team_boxscores_2017_2018.csv'

In [4]:
import pandas as pd
player_df = pd.read_csv(player_path)
team_df = pd.read_csv(team_path)
print(player_df.columns, '\n\n',team_df.columns)

Index(['id', 'created_on', 'updated_on', 'source', 'is_duplicated',
       'display_name', 'match_id', 'team_id', 'team_abbr', 'team_city',
       'player_id', 'player_name', 'start_position', 'comment', 'min', 'fgm',
       'fga', 'fg_pct', 'fg3m', 'fg3a', 'fg3_pct', 'ftm', 'fta', 'ft_pct',
       'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'to', 'pf', 'pts',
       'plus_minus', 'ast_pct', 'ast_ratio', 'ast_tov', 'blka', 'def_rating',
       'dreb_pct', 'e_def_rating', 'e_net_rating', 'e_off_rating', 'e_pace',
       'e_usg_pct', 'efg_pct', 'net_rating', 'off_rating',
       'opp_pts_2nd_chance', 'opp_pts_fb', 'opp_pts_off_tov', 'opp_pts_paint',
       'oreb_pct', 'pace', 'pfd', 'pie', 'pts_2nd_chance', 'pts_fb',
       'pts_off_tov', 'pts_paint', 'reb_pct', 'tm_tov_pct', 'ts_pct',
       'usg_pct', 'granularity', 'per_mode', 'season_id', 'dd2', 'fp', 'gp',
       'td3', 'hometeam_id', 'hometeam_abbr', 'visitorteam_id',
       'visitorteam_abbr', 'final_hscore', 'final_vscore',
     

In [5]:
# dataframe property
print(player_df.shape, team_df.shape)

(69863, 81) (5538, 62)


In [6]:
# filter match ids
match_id_field = list(player_df['match_id']) + list(team_df['match_id'])
unwanted_ids = [i for i in match_id_field if str(i)[0] != '2']
print("unwanted match_id number: ", len(unwanted_ids))

unwanted match_id number:  9665


In [7]:
# assumption testing -> match id is always 8 digits
match_id_len = [len(str(i)) == 8 for i in match_id_field]
print("Assumption is tested to be: ", all(match_id_len))

Assumption is tested to be:  True


In [0]:
# regular season match ids start with 2
# Filter all those starting with 2
def remove_non_nba(data):
  return data[(data['match_id'] >= 20000000) & (data['match_id'] < 30000000)]

In [9]:
# test match_id filter
nba_player_df = remove_non_nba(player_df)  # nba df is all match within nba(match id starts with 2)
nba_team_df = remove_non_nba(team_df)
print("Rows filtered out: ", (player_df.shape[0] + team_df.shape[0]) - (nba_player_df.shape[0] + nba_team_df.shape[0]))


Rows filtered out:  9665


In [10]:
# Now we try to understand the season data
season_id_field = list(set(player_df['season'])) + list(set(team_df['season_id']))
print("Season ids are: ", season_id_field)

Season ids are:  ['2017-18', '2018-19', 22017, 22018]


In [0]:
# convert season_id for consistency
pd.options.mode.chained_assignment = None

nba_player_df.loc[nba_player_df['season'] == '2018-19', 'season_id'] = 22018
nba_player_df.loc[nba_player_df['season'] == '2017-18', 'season_id'] = 22017
nba_player_df = nba_player_df.astype({'season_id':int})

In [0]:
# The data contains 2 seasons
# Separate 2017 and 2018 season
# Look at season_id attribute
def separate_season(data):
  return data[data['season_id'] == 22017], data[data['season_id'] == 22018]


In [14]:
# separate data based on season
player17_df, player18_df = separate_season(nba_player_df)
team17_df, team18_df = separate_season(nba_team_df)
print(player17_df.shape, player18_df.shape, team17_df.shape, team18_df.shape)

(30020, 81) (30796, 81) (2460, 62) (2460, 62)


In [0]:
# add a home/ away column to team df
def bool_home_away(player, team):
  # Use the players dataset
  # Compare team_id with hometeam_id
  # Then use match_ids in the two datasets to take home/away boolean
  # from players dataset to team dataset
  isHome = player['team_id'] == player['hometeam_id']
  player = player.assign(isHome = isHome.values)
  team = team.merge(player[['match_id', 'team_id', 'isHome']], on=['match_id', 'team_id'], how = 'left')
  team.drop_duplicates(inplace=True)
  return player, team

In [56]:
# add isHome to both player and team df
player17, team17 = bool_home_away(player17_df, team17_df)
player18, team18 = bool_home_away(player18_df, team18_df)
print(player17.shape, player18.shape, team17.shape, team18.shape)

(30020, 82) (30796, 82) (2460, 63) (2460, 63)


In [0]:
def pts_scored_recieved(player):
  player['pts_scored'] = 0 
  player['pts_received'] = 0
  player.loc[player['isHome'] == True, ['pts_scored']] = player['final_hscore'] # points scored = home score for home teams.
  player.loc[player['isHome'] == True, ['pts_received']] = player['final_vscore'] # points received = away score for home teams.
  player.loc[player['isHome'] == False, ['pts_scored']] = player['final_vscore']
  player.loc[player['isHome'] == False, ['pts_received']] = player['final_hscore']
  return player

In [60]:
pts_scored_recieved(player17)
pts_scored_recieved(player18)
player17.head()

,id,created_on,updated_on,source,is_duplicated,display_name,match_id,team_id,team_abbr,team_city,player_id,player_name,start_position,comment,min,fgm,fga,fg_pct,fg3m,fg3a,fg3_pct,ftm,fta,ft_pct,oreb,dreb,reb,ast,stl,blk,to,pf,pts,plus_minus,ast_pct,ast_ratio,ast_tov,blka,def_rating,dreb_pct,...,e_usg_pct,efg_pct,net_rating,off_rating,opp_pts_2nd_chance,opp_pts_fb,opp_pts_off_tov,opp_pts_paint,oreb_pct,pace,pfd,pie,pts_2nd_chance,pts_fb,pts_off_tov,pts_paint,reb_pct,tm_tov_pct,ts_pct,usg_pct,granularity,per_mode,season_id,dd2,fp,gp,td3,hometeam_id,hometeam_abbr,visitorteam_id,visitorteam_abbr,final_hscore,final_vscore,player_position_code,player_height,player_weight,season,isHome,pts_scored,pts_received
0,63226,2018-12-06 17:20:21,2018-12-06 17:20:21,https://stats.nba.com,NaN,NaN,21700366,1610612745,HOU,Houston,2772,Trevor Ariza,F,NaN,35.233333,4,6,0.667,2,4,0.500,0,0,0.0,0,4,4,1,2,1,1,1,10,4,0.040,12.5,1.0,0,111.4,0.143,...,0.090,0.833,2.5,113.9,11,11,19,36,0.000,96.73,0,0.101,0,7,7,4,0.067,12.5,0.833,0.086,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612762,UTA,1610612745,HOU,101,112,F-G,2.03,97.5,2017-18,False,112,101
1,63227,2018-12-06 17:20:21,2018-12-06 17:20:21,https://stats.nba.com,NaN,NaN,21700366,1610612745,HOU,Houston,201583,Ryan Anderson,F,NaN,35.150000,9,11,0.818,5,6,0.833,0,0,0.0,3,3,6,2,1,0,1,4,23,12,0.105,14.3,2.0,0,102.7,0.088,...,0.147,1.045,16.4,119.2,8,10,20,30,0.091,99.69,0,0.178,7,0,4,8,0.090,7.1,1.045,0.146,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612762,UTA,1610612745,HOU,101,112,F,2.08,108.9,2017-18,False,112,101
2,63228,2018-12-06 17:20:21,2018-12-06 17:20:21,https://stats.nba.com,NaN,NaN,21700366,1610612745,HOU,Houston,203991,Clint Capela,C,NaN,17.366667,3,6,0.500,0,0,0.000,1,5,0.2,3,5,8,0,1,1,1,5,7,9,0.000,0.0,0.0,1,82.9,0.263,...,0.246,0.500,32.3,115.2,5,4,4,16,0.167,93.97,3,0.036,0,2,0,6,0.216,11.1,0.427,0.243,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612762,UTA,1610612745,HOU,101,112,C,2.08,108.9,2017-18,False,112,101
3,63229,2018-12-06 17:20:21,2018-12-06 17:20:21,https://stats.nba.com,NaN,NaN,21700366,1610612745,HOU,Houston,201935,James Harden,G,NaN,35.150000,8,19,0.421,4,12,0.333,9,9,1.0,0,6,6,3,1,0,3,2,29,12,0.150,10.7,1.0,0,102.7,0.176,...,0.306,0.526,16.4,119.2,8,10,20,30,0.000,99.69,8,0.174,4,0,2,6,0.090,10.7,0.632,0.305,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612762,UTA,1610612745,HOU,101,112,G,1.96,99.8,2017-18,False,112,101
4,63230,2018-12-06 17:20:21,2018-12-06 17:20:21,https://stats.nba.com,NaN,NaN,21700366,1610612745,HOU,Houston,101108,Chris Paul,G,NaN,36.716667,7,14,0.500,2,5,0.400,2,2,1.0,1,8,9,13,3,1,2,3,18,12,0.520,43.3,6.5,0,102.7,0.267,...,0.214,0.571,16.2,118.9,9,13,11,36,0.029,96.74,5,0.221,2,2,2,8,0.141,6.7,0.605,0.207,player_game,PerGame,22017,NaN,NaN,NaN,NaN,1610612762,UTA,1610612745,HOU,101,112,G,1.83,79.4,2017-18,False,112,101
